In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import json
import os

In [ ]:
mask = np.zeros([img.shape[0], img.shape[1], 1], dtype=np.uint8)

for contour in contours:
    contour = np.array(contour)
    xpoints = contour.transpose()[0][0]
    ypoints = contour.transpose()[1][0]
    
    rr, cc = skimage.draw.polygon(xpoints, ypoints)
    
    mask[cc, rr, 0] = img[cc, rr, 0]
    
cv.imshow('asdf', mask)
cv.waitKey(0)
cv.destroyAllWindows()

In [18]:
img = cv2.imread("datasets/test_mask/mask_Object1_3.png")
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret, thresh = cv2.threshold(imgray, 127, 255, 0)

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

all_xpoints = []
all_ypoints = []

for contour in contours:
    xpoints = contour.transpose()[0][0]
    ypoints = contour.transpose()[1][0]
    
    all_xpoints.append(xpoints)
    all_ypoints.append(ypoints)

# cv2.imshow('test', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[array([320, 319, 312, 311, 305, 304, 298, 297, 290, 289, 283, 282, 276,
       275, 269, 268, 264, 265, 265, 266, 266, 267, 267, 268, 284, 285,
       301, 302, 318, 319, 324], dtype=int32), array([393, 392, 385, 384, 378, 377, 385, 385, 388, 390, 391, 392, 394,
       394, 395, 395, 396, 396, 399, 399, 402, 402, 403, 403, 388, 387,
       384, 384, 385, 385, 386, 386, 387, 387, 386, 386, 385, 385, 384,
       384, 380, 379, 378, 377, 376, 375, 374, 369, 368, 362, 361, 354,
       353, 347, 346, 341, 338, 337, 321, 320, 304, 303, 298, 300, 301,
       307, 308, 314, 315, 322, 323, 329, 330, 336, 337, 344, 345, 351,
       352, 358, 359, 365, 366, 373, 374, 380, 381, 387, 388, 395, 396,
       402, 403, 409, 410, 413, 414, 415, 416, 417, 421, 421, 422, 422,
       423, 423, 422, 422, 421, 421, 420, 420, 419, 419, 418, 418, 417,
       417, 416, 416, 415, 415, 414, 414, 413, 413, 412, 412, 411, 411,
       410, 410, 408, 408, 407, 406, 404, 403, 402, 400, 399], dtype=int32)]
2


In [38]:
def find_contours(img):
    # Grayscale image 
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)

    # Find the contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Return an empty array if it did not find any contours
    if len(contours) == 0:
        return np.array([])
    
    all_xpoints = []
    all_ypoints = []

    for contour in contours:
        xpoints = contour.transpose()[0][0]
        ypoints = contour.transpose()[1][0]

        all_xpoints.append(xpoints.tolist())
        all_ypoints.append(ypoints.tolist())

    return all_xpoints, all_ypoints

In [33]:
class Mask(object):
    def __init__(self, file):
        self.name = file[0].split('Object')[1].split('.')[0]
        self.classification = self.name.split('_')[0]
        self.contours = find_contours(cv2.imread(file[0]))


class Image(object):
    def __init__(self, file):
        print(file)
        self.image = cv2.imread(file)
        self.name = file.split('/')[3]
        self.mask = self.get_mask()

    def get_mask(self):
        path = "./datasets/test_mask/mask_Object" + self.name.split('_Object')[1]
        return Mask(glob.glob(path))

In [40]:
# Save the points to a json file
images = []

for file in glob.glob("./datasets/test_image/*.png"):
    images.append(Image(file))
    
data = {}

for image in images:
    region = {}
    
    region['name'] = image.mask.name
    region['class'] = image.mask.classification

    x_points, y_points = image.mask.contours
    
    region['all_points_x'] = x_points
    region['all_points_y'] = y_points
    
    image_information = {}
    height, width, _ = image.image.shape
    image_information['filename'] = image.name
    image_information['width'] = width
    image_information['height'] = height
    image_information['region'] = region
    
    data['image_' + image.name] = image_information
    
with open('datasets/test_image/annotations.json', 'w') as json_file:
    json.dump(data, json_file)

./datasets/test_image/image_Object1_3.png


In [8]:
#For testing
import random

annotations = json.load(open(os.path.join("test_image/annotations.json")))
annotations = list(annotations.values())

annotation = annotations[0]

fin_img = cv2.imread("./test_image/" + annotation['filename'])

x_points = annotation['region']['all_points_x']
y_points = annotation['region']['all_points_y']
# print(len(y_points) == len(x_points))

color = random.choice([[0,0,255], [255, 0, 0], [0, 255, 0], [255, 255, 0], [0, 255, 255], [255, 0, 255]])

points = np.array([x_points, y_points]).transpose()

cv2.polylines(fin_img, [points], True, color, 2)

# Show
cv2.imshow(annotation['filename'], fin_img)

cv2.waitKey(0)
cv2.destroyAllWindows()